### Importing Packages 

In [6]:
import pickle

import numpy as np 
import pandas as pd

import re 
import string 

import nltk
#from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer 

from langdetect import detect

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
# from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import gensim
from gensim import corpora, models, similarities, matutils

## Opening Scrapped Data 

In [ ]:
with open('pokemon.pickle','rb') as read_file:
    df = pickle.load(read_file)

# Data Cleaning 

## Dataset 

In [ ]:
df.info()  

In [ ]:
df['rating'] = df['rating'].astype(int)

In [ ]:
### let's group the reviews by their ratings 
### following metacritics, 0 to 4 = negative, 5 to 7 mixed, 8 and above = positive 

def sentiment(x):
    if x > 7:
        return 'positive'
    if x < 5:
        return 'negative'
    else: return 'mixed'

df['sentiment'] = df['rating'].apply(lambda x:sentiment(x))


In [ ]:
df['sentiment'].value_counts(normalize = True)

In [ ]:
### Some users posted on both pokemon sword and shield. 
### Did some checks, these reviews were the same  

df[df.duplicated('name')]

In [ ]:
df.review[df['name'] == 'Mack_thge_Sack']

In [ ]:
df.reset_index(inplace = True, drop = True)

In [ ]:
### dropping duplicate names 
df.drop_duplicates(subset='name', keep = 'first', inplace = True)

In [ ]:
### detect review language and returns NaN if not english 
def language_detection(x): 
    result = detect(x)
    if result == 'en':
        return x 
    else: return np.NaN 
    
df['review'] = df['review'].apply(lambda x:language_detection(x))

### drop reviews that are not in english 
df.dropna(inplace = True)

In [ ]:
#df.to_pickle('clean_dataset.pickle')

### Text Cleaning 

In [18]:
with open('clean_dataset.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [19]:
df['review'][4]

"As a lifelong fan of Pokemon games, I was ecstatic when a new mainline Pokemon game was announced for the Switch, a home console! Pokemon Gen 7 was alright, but a little too slow and easy for my tastes. But for Gen 8 all I was expecting was a nice polished HD Pokemon game, I didn't need an open world or anything as it is Gamefreak's first home console Pokemon title, after all. But no, that isn't what we got. We got an unfinished game, with over half of the national Pokedex not in the game, choppy, reused animations, reused models when they said they were brand new, even easier difficulty, a significantly shorter campaign as well as next to no post-game, not to mention many other features removed. All I have to say is, why? Pokemon is the highest most profitable franchise of all time, and Nintendo has been hitting it out of the park lately with great game after great game, like Zelda, Mario etc. So why can't Gamefreak improve Pokemon too? It's only gotten worse with time. I hope Ninten

In [20]:
### remove stopwords text, using this method just incase I want to add more stopwords 
nltk_stop_words = nltk.corpus.stopwords.words('english')

stop_words =  list(STOP_WORDS)
stop_words.extend(['game','pokemon','pokémon'])

for word in nltk_stop_words:
    if word in stop_words: 
        continue
    else: stop_words.append(word)

# for word in stop_words:
#     no_punct = remove_punctuation(word)
#     if no_punct not in stop_words: 
#         stop_words.append(no_punct)

In [21]:
def make_lower(text):
    return text.lower()

def remove_words(text,wordlist):
    for word in wordlist:
        if word in text.split():
            text = re.sub(r'\b{}\b'.format(word), '', text)  
    return text

def remove_digits(text):
    return re.sub('\d', ' ', text)

def remove_punctuation(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) 
    return re.sub(r'[^\w\s]', ' ', text)

def strip_extraspace(text):
    return re.sub('\s\s+',' ', text)

def replace_word(text,word,replacement):
    return text.replace(word,replacement)

def remove_r(text):
    return text.replace('\r',' ')
#df['review'] = df['review'].apply(lambda x:remove_punctuation(x))

def clean_text(text):
    text = make_lower(text)
    text = remove_punctuation(text)
    text = remove_digits(text)
    text = replace_word(text,'game freak','gamefreak')
    text = replace_word(text, 'game play', 'gameplay')
    text = remove_words(text,stop_words)
    text = remove_r(text)
    text = strip_extraspace(text)
    return text

In [22]:
df['review'] = df['review'].apply(lambda x:clean_text(x))

In [23]:
df['review'][5]

' routes virtually difficulty endgame battle tower modes beatable hour roughly hour story dungeons average graphics world barely interacts objects especially wild area course pokedex purge new designs leave alot desired designs genuinely good forget humanoid starter evolves humanoid simple designs eiscue importantly man sky level damage control gamefreak instead listening customer base fans long know generally want games need look masterpieces hg ss know direction follow good endgame following wish gamefreak include difficulty options games unlockable clear grew games longer challenge add difficult new big disappointment gen sinnoh remakes complete swsh likely garbage '

In [ ]:
#df['review'] = df['review'].apply(lambda x:remove_stopwords(str.split(x)))

In [24]:
# lemmatizer=WordNetLemmatizer()

# def lemmatize_words(x):
#     lemmed_string = ''
#     for word in x.split():
#         lemmed_string = lemmed_string+' '+lemmatizer.lemmatize(word)  
#     return lemmed_string.lstrip()

sp = spacy.load('en_core_web_sm')

def lemmatize_words(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    text = sp(text)
    lemmed_string =''
    for word in text:
        if word.pos_ in allowed_postags:
            if word.lemma_ == '-PRON-' or word.lemma_ in stop_words:
                #word.lemma_ = word.orth_  
                continue 
            else: lemmed_string = lemmed_string+' '+word.lemma_
    return lemmed_string.lstrip()

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [25]:
df['review'] = df['review'].apply(lambda x:lemmatize_words(x, allowed_postags=['NOUN', 'VERB']))

In [26]:
# # Build the bigram and trigram models - reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# data = []
# for entry in df.review:
#     data.append(entry.split())

# bigram = gensim.models.Phrases(data, min_count=5, threshold=10) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data], threshold=10)  

# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# def make_bigrams(x):
#     text = bigram_mod[x.split()] 
#     grammed_string = ''
#     for word in text: 
#         grammed_string = grammed_string+' '+word 
#     return grammed_string 

# def make_trigrams(x):
#     text = trigram_mod[bigram_mod[x.split()] ] 
#     grammed_string = ''
#     for word in text: 
#         grammed_string = grammed_string+' '+word 
#     return grammed_string 

In [27]:
# df['review1'] = df['review'].apply(lambda x:make_bigrams(x))  
# df['review1'] = df['review'].apply(lambda x:make_trigrams(x))  

In [28]:
df['review'][55]

'let way fan disappoint betray fan lie consumer problem plague franchise story write character pace villain team generation brain play charge cut content nee write read review know talk people support gamefreak'

In [29]:
df.to_pickle('dfclean_b4rare.pickle') ### cleaned text and lemmatised 

In [ ]:
with open('dfclean_b4rare.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [33]:
# ### create list of rare words by filtering on word counts
freq = pd.DataFrame(df.review.str.split(expand=True).stack().value_counts())
freq = freq[freq<6]
freq.dropna(inplace = True)
freq.reset_index(inplace = True)
freq = freq['index'].to_list()

In [34]:
freq

['favour',
 'hairstyle',
 'breathtake',
 'compose',
 'gigantamax',
 'non',
 'slide',
 'stamp',
 'nitpick',
 'cent',
 'awaken',
 'priority',
 'healing',
 'street',
 'boil',
 'criticize',
 'fav',
 'inability',
 'girl',
 'merit',
 'cancel',
 'xp',
 'gap',
 'consequence',
 'custom',
 'response',
 'bet',
 'professor',
 'absolute',
 'basic',
 'recognition',
 'absence',
 'spinoff',
 'recognize',
 'pkm',
 'distract',
 'bro',
 'dog',
 'entrie',
 'random',
 'v',
 'purchasing',
 'success',
 'excellent',
 'variation',
 'poke',
 'great',
 'idiot',
 'category',
 'sonority',
 'circle',
 'eeveelution',
 'input',
 'interface',
 'chronicle',
 'motion',
 'outlet',
 'spending',
 'finding',
 'single',
 'light',
 'slow',
 'eternatus',
 'barrier',
 'lobby',
 'shiny',
 'status',
 'pet',
 'simplicity',
 'autosave',
 'episode',
 'immerse',
 'th',
 'food',
 'polishing',
 'soulsilver',
 'toy',
 'hole',
 'ooze',
 'debut',
 'white',
 'book',
 'witness',
 'host',
 'satisfaction',
 'justification',
 'exact',
 'icon',

In [35]:
df['review'] = df['review'].apply(lambda x:remove_words(x,freq))  

In [30]:
# ### checking high frequency words to add to stopword list 
freq = pd.DataFrame(df.review.str.split(expand=True).stack().value_counts())

In [32]:
pd.set_option('display.max_rows', 1000)
freq.tail(100)

,0
weed,1
mask,1
pokeshop,1
hazy,1
persist,1
clown,1
pokemonmon,1
receiving,1
battke,1
monetization,1


In [36]:
df.to_pickle('dfclean.pickle')

# with open('dfclean.pickle','rb') as read_file:
#      df = pickle.load(read_file)

In [ ]:
#freq = pd.DataFrame(df.review.str.split(expand=True).stack().value_counts())

## Splitting df by sentiment 

In [37]:
negative = df[df['sentiment']=='negative']
mixed = df[df['sentiment']=='mixed']
positive = df[df['sentiment']=='postive']


In [38]:
negative

,name,date,rating,review,game,sentiment
0,Otonaburu,"Nov 15, 2019",4,hardware feel port feature improvement come pr...,shield,negative
1,Metagrass,"Nov 15, 2019",2,sword version copy imagine bother year fan reg...,shield,negative
2,Drakhis,"Nov 15, 2019",2,fan lot problem turn exp bug wipe switch versi...,shield,negative
3,Ok_Then,"Nov 15, 2019",4,feel port build fan enjoy feel generation upg...,shield,negative
4,NintendoGuy64,"Nov 15, 2019",0,fan announce console alright taste need world...,shield,negative
...,...,...,...,...,...,...
3224,moosbuckel,"Nov 21, 2019",1,play hour bore story route bore way exp syste...,sword,negative
3227,Bashbach1,"Nov 21, 2019",4,know begin gamefreak run idea remodel remade v...,sword,negative
3233,Eduardobritola,"Nov 22, 2019",3,world design experience old,sword,negative
3250,Potteriti,"Nov 24, 2019",2,design let,sword,negative


In [39]:
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range = (1,2),
                                   strip_accents = 'ascii', 
                                   token_pattern="\\b[a-z][a-z][a-z]+\\b")

In [40]:
doc_word = vectorizer.fit_transform(negative.review)
doc_word.shape

(1349, 29283)

In [ ]:
pd.DataFrame(doc_word.toarray(), index=negative.review, columns=vectorizer.get_feature_names()).head(10)

## LSA 

In [41]:
lsa = TruncatedSVD(4)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.08084704, 0.02215631, 0.0150878 , 0.01223327])

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

In [42]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [43]:
display_topics(lsa, vectorizer.get_feature_names(), 15)


Topic  0
feel, battle, animation, time, area, look, story, play, cut, model, character, feature, fan, switch, thing

Topic  1
feel, area, battle, gym, story, route, fight, explore, catch, challenge, character, design, level, team, raid

Topic  2
animation, battle, model, look, attack, texture, turn, model animation, tree, overworld, animation model, reuse, place, trainer, pop

Topic  3
area, gym, cut, catch, team, problem, level, character, remove, find, fight, way, reason, content, turn


### NMF 

In [44]:
nmf_model = NMF(4)
doc_topic = nmf_model.fit_transform(doc_word)

In [ ]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

In [45]:
display_topics(nmf_model, vectorizer.get_feature_names(), 15)


Topic  0
area, gym, battle, story, character, catch, team, time, level, problem, route, find, point, fight, way

Topic  1
fan, play, time, content, cut, franchise, gamefreak, switch, release, want, graphic, year, thing, people, buy

Topic  2
animation, model, look, cut, battle, quality, turn, lie, gamefreak, graphic, lot, let, texture, switch, time

Topic  3
feel, battle, look, story, time, play, design, area, world, lack, lot, player, entry, music, compare


### LDA 

In [46]:
vectorizer.fit(negative.review)
doc_word = vectorizer.transform(negative.review).transpose()

In [47]:
corpus = matutils.Sparse2Corpus(doc_word)

In [48]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [49]:
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

In [52]:
#lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=20)

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=77,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50, iterations = 100,
                                           alpha='auto',
                                           per_word_topics=True)

2019-11-30 03:43:27,789 : INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2019-11-30 03:43:27,793 : INFO : using symmetric eta at 0.25
2019-11-30 03:43:27,801 : INFO : using serial LDA version on this node
2019-11-30 03:43:27,817 : INFO : running online (multi-pass) LDA training, 4 topics, 50 passes over the supplied corpus of 1349 documents, updating model once every 100 documents, evaluating perplexity every 1000 documents, iterating 100x with a convergence threshold of 0.001000
2019-11-30 03:43:27,825 : INFO : PROGRESS: pass 0, at document #100/1349
2019-11-30 03:43:28,025 : INFO : optimized alpha [0.2403351, 0.19001815, 0.18777363, 0.25144565]
2019-11-30 03:43:28,033 : INFO : merging changes from 100 documents into a model of 1349 documents
2019-11-30 03:43:28,045 : INFO : topic #0 (0.240): 0.007*"animation" + 0.005*"battle" + 0.005*"look" + 0.005*"cut" + 0.005*"time" + 0.004*"story" + 0.004*"area" + 0.004*"franchise" + 0.004*"quality" + 0.004*"graphic"
2019-11

2019-11-30 03:43:29,105 : INFO : topic #2 (0.276): 0.012*"cut" + 0.009*"animation" + 0.007*"feel" + 0.006*"content" + 0.005*"lie" + 0.005*"fan" + 0.005*"play" + 0.004*"rush" + 0.004*"lack" + 0.004*"look"
2019-11-30 03:43:29,109 : INFO : topic #3 (0.306): 0.008*"play" + 0.007*"graphic" + 0.007*"feel" + 0.007*"animation" + 0.005*"time" + 0.005*"switch" + 0.005*"area" + 0.005*"look" + 0.005*"battle" + 0.005*"story"
2019-11-30 03:43:29,109 : INFO : topic diff=0.477432, rho=0.377964
2019-11-30 03:43:29,117 : INFO : PROGRESS: pass 0, at document #800/1349
2019-11-30 03:43:29,253 : INFO : optimized alpha [0.30279762, 0.2228276, 0.30424356, 0.32791957]
2019-11-30 03:43:29,261 : INFO : merging changes from 100 documents into a model of 1349 documents
2019-11-30 03:43:29,273 : INFO : topic #0 (0.303): 0.008*"feel" + 0.007*"franchise" + 0.007*"look" + 0.006*"animation" + 0.005*"story" + 0.005*"time" + 0.005*"area" + 0.005*"cut" + 0.005*"graphic" + 0.004*"year"
2019-11-30 03:43:29,277 : INFO : top

2019-11-30 03:43:30,661 : INFO : topic #1 (0.272): 0.007*"area" + 0.005*"time" + 0.005*"hour" + 0.004*"story" + 0.004*"player" + 0.004*"battle" + 0.004*"character" + 0.004*"design" + 0.003*"region" + 0.003*"city"
2019-11-30 03:43:30,661 : INFO : topic #2 (0.471): 0.011*"cut" + 0.009*"animation" + 0.008*"content" + 0.007*"fan" + 0.007*"lie" + 0.005*"model" + 0.005*"gamefreak" + 0.005*"feel" + 0.004*"rush" + 0.004*"company"
2019-11-30 03:43:30,665 : INFO : topic #3 (0.645): 0.008*"area" + 0.008*"feel" + 0.008*"battle" + 0.008*"play" + 0.007*"animation" + 0.007*"look" + 0.006*"story" + 0.006*"feature" + 0.006*"graphic" + 0.006*"time"
2019-11-30 03:43:30,669 : INFO : topic diff=0.432443, rho=0.267261
2019-11-30 03:43:30,677 : INFO : PROGRESS: pass 1, at document #100/1349
2019-11-30 03:43:30,801 : INFO : optimized alpha [0.5034309, 0.2491722, 0.42505723, 0.6233885]
2019-11-30 03:43:30,813 : INFO : merging changes from 100 documents into a model of 1349 documents
2019-11-30 03:43:30,825 : I

2019-11-30 03:43:31,573 : INFO : topic #1 (0.245): 0.006*"area" + 0.005*"hour" + 0.005*"time" + 0.003*"region" + 0.003*"story" + 0.003*"character" + 0.003*"design" + 0.003*"battle" + 0.003*"fan" + 0.003*"want"
2019-11-30 03:43:31,577 : INFO : topic #2 (0.431): 0.014*"cut" + 0.010*"animation" + 0.008*"lie" + 0.007*"content" + 0.007*"fan" + 0.005*"gamefreak" + 0.005*"model" + 0.004*"rush" + 0.004*"quality" + 0.004*"feel"
2019-11-30 03:43:31,577 : INFO : topic #3 (0.641): 0.009*"play" + 0.008*"feel" + 0.008*"area" + 0.008*"animation" + 0.007*"battle" + 0.007*"graphic" + 0.007*"time" + 0.006*"look" + 0.006*"story" + 0.005*"switch"
2019-11-30 03:43:31,581 : INFO : topic diff=0.373112, rho=0.254082
2019-11-30 03:43:31,589 : INFO : PROGRESS: pass 1, at document #800/1349
2019-11-30 03:43:31,661 : INFO : optimized alpha [0.48119983, 0.24610174, 0.4474231, 0.65277964]
2019-11-30 03:43:31,673 : INFO : merging changes from 100 documents into a model of 1349 documents
2019-11-30 03:43:31,685 : INF

2019-11-30 03:43:32,605 : INFO : optimized alpha [0.5925609, 0.24036688, 0.4834428, 0.9036224]
2019-11-30 03:43:32,613 : INFO : merging changes from 49 documents into a model of 1349 documents
2019-11-30 03:43:32,625 : INFO : topic #0 (0.593): 0.009*"franchise" + 0.008*"feel" + 0.008*"look" + 0.007*"story" + 0.005*"design" + 0.005*"time" + 0.005*"fan" + 0.005*"world" + 0.005*"lack" + 0.005*"year"
2019-11-30 03:43:32,629 : INFO : topic #1 (0.240): 0.005*"area" + 0.004*"hour" + 0.004*"player" + 0.004*"time" + 0.003*"character" + 0.003*"city" + 0.003*"story" + 0.003*"region" + 0.003*"design" + 0.002*"team"
2019-11-30 03:43:32,633 : INFO : topic #2 (0.483): 0.013*"cut" + 0.009*"animation" + 0.009*"lie" + 0.008*"fan" + 0.008*"content" + 0.006*"gamefreak" + 0.006*"model" + 0.005*"graphic" + 0.004*"buy" + 0.004*"rush"
2019-11-30 03:43:32,633 : INFO : topic #3 (0.904): 0.010*"area" + 0.009*"feel" + 0.009*"battle" + 0.008*"animation" + 0.008*"play" + 0.008*"look" + 0.007*"story" + 0.007*"time" 

...

2019-11-30 03:44:30,692 : INFO : topic #1 (0.038): 0.001*"capture" + 0.001*"join" + 0.001*"road" + 0.001*"save" + 0.001*"block" + 0.001*"frame" + 0.001*"theme" + 0.001*"breed" + 0.001*"sound" + 0.001*"chairman"
2019-11-30 03:44:30,692 : INFO : topic #2 (0.119): 0.009*"lie" + 0.008*"cut" + 0.006*"fan" + 0.005*"gamefreak" + 0.004*"model" + 0.003*"buy" + 0.002*"gamefreak lie" + 0.002*"half" + 0.002*"money" + 0.002*"company"
2019-11-30 03:44:30,696 : INFO : topic #3 (0.826): 0.010*"animation" + 0.010*"feel" + 0.008*"look" + 0.008*"area" + 0.008*"time" + 0.008*"story" + 0.007*"battle" + 0.007*"play" + 0.006*"graphic" + 0.005*"switch"
2019-11-30 03:44:30,696 : INFO : topic diff=0.107272, rho=0.127526
2019-11-30 03:44:30,704 : INFO : PROGRESS: pass 48, at document #100/1349
2019-11-30 03:44:30,740 : INFO : optimized alpha [0.085413255, 0.03823958, 0.11748814, 0.8365382]
2019-11-30 03:44:30,748 : INFO : merging changes from 100 documents into a model of 1349 documents
2019-11-30 03:44:30,764 :

2019-11-30 03:44:31,172 : INFO : topic #0 (0.084): 0.004*"franchise" + 0.002*"fan" + 0.002*"quality" + 0.002*"money" + 0.002*"history" + 0.001*"cash" + 0.001*"company" + 0.001*"cash grab" + 0.001*"waste" + 0.001*"grab"
2019-11-30 03:44:31,172 : INFO : topic #1 (0.038): 0.001*"transfer" + 0.001*"join" + 0.001*"save" + 0.001*"block" + 0.001*"frame drop" + 0.001*"frame" + 0.001*"man" + 0.001*"learn" + 0.000*"snap" + 0.000*"skill"
2019-11-30 03:44:31,176 : INFO : topic #2 (0.121): 0.009*"cut" + 0.009*"lie" + 0.006*"fan" + 0.005*"gamefreak" + 0.003*"model" + 0.003*"buy" + 0.002*"gamefreak lie" + 0.002*"quality" + 0.002*"company" + 0.002*"content"
2019-11-30 03:44:31,176 : INFO : topic #3 (0.838): 0.010*"feel" + 0.010*"animation" + 0.008*"time" + 0.008*"look" + 0.008*"area" + 0.008*"story" + 0.007*"play" + 0.007*"battle" + 0.007*"graphic" + 0.006*"cut"
2019-11-30 03:44:31,180 : INFO : topic diff=0.108365, rho=0.126501
2019-11-30 03:44:31,184 : INFO : PROGRESS: pass 48, at document #800/1349


2019-11-30 03:44:31,728 : INFO : topic diff=0.114587, rho=0.126501
2019-11-30 03:44:31,796 : INFO : -9.308 per-word bound, 633.8 perplexity estimate based on a held-out corpus of 49 documents with 3655 words
2019-11-30 03:44:31,800 : INFO : PROGRESS: pass 48, at document #1349/1349
2019-11-30 03:44:31,816 : INFO : optimized alpha [0.08524549, 0.038089886, 0.11848508, 0.82553214]
2019-11-30 03:44:31,828 : INFO : merging changes from 49 documents into a model of 1349 documents
2019-11-30 03:44:31,840 : INFO : topic #0 (0.085): 0.004*"franchise" + 0.003*"fan" + 0.002*"quality" + 0.002*"cash" + 0.002*"company" + 0.001*"money" + 0.001*"cash grab" + 0.001*"comparison" + 0.001*"grab" + 0.001*"franchise world"
2019-11-30 03:44:31,840 : INFO : topic #1 (0.038): 0.001*"capture" + 0.001*"join" + 0.001*"road" + 0.001*"save" + 0.001*"block" + 0.001*"frame" + 0.001*"theme" + 0.001*"breed" + 0.001*"sound" + 0.001*"chairman"
2019-11-30 03:44:31,844 : INFO : topic #2 (0.118): 0.009*"lie" + 0.008*"cut" 

2019-11-30 03:44:32,272 : INFO : topic #3 (0.826): 0.010*"feel" + 0.010*"animation" + 0.008*"time" + 0.008*"look" + 0.008*"story" + 0.008*"area" + 0.007*"play" + 0.007*"battle" + 0.007*"graphic" + 0.006*"cut"
2019-11-30 03:44:32,272 : INFO : topic diff=0.089092, rho=0.125501
2019-11-30 03:44:32,276 : INFO : PROGRESS: pass 49, at document #700/1349
2019-11-30 03:44:32,312 : INFO : optimized alpha [0.08338128, 0.038207848, 0.120439865, 0.8377601]
2019-11-30 03:44:32,324 : INFO : merging changes from 100 documents into a model of 1349 documents
2019-11-30 03:44:32,336 : INFO : topic #0 (0.083): 0.004*"franchise" + 0.002*"fan" + 0.002*"quality" + 0.002*"money" + 0.001*"cash" + 0.001*"history" + 0.001*"company" + 0.001*"cash grab" + 0.001*"waste" + 0.001*"grab"
2019-11-30 03:44:32,340 : INFO : topic #1 (0.038): 0.001*"transfer" + 0.001*"join" + 0.001*"save" + 0.001*"block" + 0.001*"frame drop" + 0.001*"frame" + 0.001*"man" + 0.001*"learn" + 0.000*"snap" + 0.000*"skill"
2019-11-30 03:44:32,3

2019-11-30 03:44:32,889 : INFO : topic #2 (0.120): 0.010*"lie" + 0.009*"cut" + 0.006*"fan" + 0.005*"gamefreak" + 0.004*"model" + 0.003*"buy" + 0.003*"company" + 0.003*"gamefreak lie" + 0.002*"half" + 0.002*"money"
2019-11-30 03:44:32,889 : INFO : topic #3 (0.833): 0.010*"animation" + 0.009*"feel" + 0.008*"time" + 0.008*"look" + 0.008*"area" + 0.007*"battle" + 0.007*"play" + 0.007*"story" + 0.007*"graphic" + 0.006*"cut"
2019-11-30 03:44:32,893 : INFO : topic diff=0.113476, rho=0.125501
2019-11-30 03:44:32,965 : INFO : -9.304 per-word bound, 631.9 perplexity estimate based on a held-out corpus of 49 documents with 3655 words
2019-11-30 03:44:32,965 : INFO : PROGRESS: pass 49, at document #1349/1349
2019-11-30 03:44:32,981 : INFO : optimized alpha [0.08506217, 0.038058955, 0.11822928, 0.82487226]
2019-11-30 03:44:32,993 : INFO : merging changes from 49 documents into a model of 1349 documents
2019-11-30 03:44:33,005 : INFO : topic #0 (0.085): 0.004*"franchise" + 0.003*"fan" + 0.002*"quali

In [53]:
lda_model.print_topics()

2019-11-30 03:44:33,025 : INFO : topic #0 (0.085): 0.004*"franchise" + 0.003*"fan" + 0.002*"quality" + 0.002*"cash" + 0.002*"company" + 0.001*"money" + 0.001*"cash grab" + 0.001*"comparison" + 0.001*"grab" + 0.001*"franchise world"
2019-11-30 03:44:33,029 : INFO : topic #1 (0.038): 0.001*"capture" + 0.001*"join" + 0.001*"road" + 0.001*"save" + 0.001*"block" + 0.001*"frame" + 0.001*"theme" + 0.001*"breed" + 0.001*"sound" + 0.001*"chairman"
2019-11-30 03:44:33,029 : INFO : topic #2 (0.118): 0.009*"lie" + 0.008*"cut" + 0.006*"fan" + 0.005*"gamefreak" + 0.004*"model" + 0.003*"buy" + 0.002*"gamefreak lie" + 0.002*"company" + 0.002*"half" + 0.002*"money"
2019-11-30 03:44:33,033 : INFO : topic #3 (0.825): 0.010*"animation" + 0.010*"feel" + 0.008*"look" + 0.008*"area" + 0.008*"time" + 0.008*"story" + 0.007*"battle" + 0.007*"play" + 0.006*"graphic" + 0.005*"switch"


[(0,
  '0.004*"franchise" + 0.003*"fan" + 0.002*"quality" + 0.002*"cash" + 0.002*"company" + 0.001*"money" + 0.001*"cash grab" + 0.001*"comparison" + 0.001*"grab" + 0.001*"franchise world"'),
 (1,
  '0.001*"capture" + 0.001*"join" + 0.001*"road" + 0.001*"save" + 0.001*"block" + 0.001*"frame" + 0.001*"theme" + 0.001*"breed" + 0.001*"sound" + 0.001*"chairman"'),
 (2,
  '0.009*"lie" + 0.008*"cut" + 0.006*"fan" + 0.005*"gamefreak" + 0.004*"model" + 0.003*"buy" + 0.002*"gamefreak lie" + 0.002*"company" + 0.002*"half" + 0.002*"money"'),
 (3,
  '0.010*"animation" + 0.010*"feel" + 0.008*"look" + 0.008*"area" + 0.008*"time" + 0.008*"story" + 0.007*"battle" + 0.007*"play" + 0.006*"graphic" + 0.005*"switch"')]

In [ ]:
vectorizer.vocabulary_.items()